# Vectara

[Vectara](https://vectara.com/) 是一个值得信赖的 AI 助手和代理平台，专注于企业级任务的关键应用就绪性。
Vectara 无服务器 RAG 即服务通过易于使用的 API 提供了 RAG 的所有组件，包括：
1. 从文件（PDF、PPT、DOCX 等）提取文本的方法。
2. 提供行业领先性能的基于 ML 的切块。
3. [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) 嵌入模型。
4. 其内部向量数据库，用于存储文本块和嵌入向量。
5. 查询服务，自动将查询编码为嵌入，并检索最相关的文本片段，包括对 [混合搜索](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) 的支持，以及多种重排选项，如 [多语言相关性重排器](https://www.vectara.com/blog/deep-dive-into-vectara-multilingual-reranker-v1-state-of-the-art-reranker-across-100-languages)、[MMR](https://vectara.com/get-diverse-results-and-comprehensive-summaries-with-vectaras-mmr-reranker/)、[UDF 重排器](https://vectara.com/blog/rag-with-user-defined-functions-based-reranking)。
6. 一个 LLM，用于基于检索到的文档（上下文）创建 [生成式摘要](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview)，包括引用。

更多信息：
- [文档](https://docs.vectara.com/docs/)
- [API Playground](https://docs.vectara.com/docs/rest-api/)
- [快速入门](https://docs.vectara.com/docs/quickstart)

本 Notebook 展示了如何使用基本检索功能，将 Vectara 用作 Vector Store（不进行摘要），包括：`similarity_search` 和 `similarity_search_with_score`，以及使用 LangChain 的 `as_retriever` 功能。


## 设置

要使用 `VectaraVectorStore`，您首先需要安装配套包。

In [ ]:
!uv pip install -U pip && uv pip install -qU langchain-vectara

# 入门指南

准备开始，请按照以下步骤操作：
1. 如果还没有，请先[注册](https://www.vectara.com/integrations/langchain)以获取免费的 Vectara 试用版。
2. 在您的账户中，您可以创建一个或多个语料库。每个语料库代表一个区域，用于在摄入输入文档时存储文本数据。要创建语料库，请使用 **“创建语料库”** 按钮。然后，为您创建的语料库提供一个名称和描述。您可以选择性地定义筛选属性并应用一些高级选项。如果您点击已创建的语料库，您可以在顶部看到其名称和语料库 ID。
3. 接下来，您需要创建 API 密钥来访问该语料库。在语料库视图中点击 **“访问控制”** 选项卡，然后点击 **“创建 API 密钥”** 按钮。为您的密钥命名，并选择是只需要查询权限还是查询+索引权限。点击“创建”，您现在就拥有了一个有效的 API 密钥。请妥善保管此密钥。

要将 LangChain 与 Vectara 结合使用，您需要获取这两个值：`corpus_key` 和 `api_key`。
您可以通过两种方式将 `VECTARA_API_KEY` 提供给 LangChain：

1. 在您的环境中包含这两个变量：`VECTARA_API_KEY`。

   例如，您可以使用 `os.environ` 和 `getpass` 设置这些变量，如下所示：

```python
import os
import getpass

os.environ["VECTARA_API_KEY"] = getpass.getpass("Vectara API Key:")
```

2. 将它们添加到 `Vectara` 向量存储构造函数中：

```python
vectara = Vectara(
    vectara_api_key=vectara_api_key
)
```

在本笔记本中，我们假设它们已在环境中提供。

In [4]:
import os

os.environ["VECTARA_API_KEY"] = "<VECTARA_API_KEY>"
os.environ["VECTARA_CORPUS_KEY"] = "VECTARA_CORPUS_KEY"

from langchain_vectara import Vectara
from langchain_vectara.vectorstores import (
    ChainReranker,
    CorpusConfig,
    CustomerSpecificReranker,
    File,
    GenerationConfig,
    MmrReranker,
    SearchConfig,
    VectaraQueryConfig,
)

vectara = Vectara(vectara_api_key=os.getenv("VECTARA_API_KEY"))

首先，我们将 state-of-the-union 文本加载到 Vectara 中。

请注意，我们使用的是 `add_files` 接口，该接口不需要任何本地处理或分块——Vectara 接收文件内容并执行所有必要的前期处理、分块和嵌入操作，将其存入其知识库。

在本例中，我们使用的是 `.txt` 文件，但对于许多其他[文件类型](https://docs.vectara.com/docs/api-reference/indexing-apis/file-upload/file-upload-filetypes)，情况也同样适用。

In [5]:
corpus_key = os.getenv("VECTARA_CORPUS_KEY")
file_obj = File(
    file_path="../document_loaders/example_data/state_of_the_union.txt",
    metadata={"source": "text_file"},
)
vectara.add_files([file_obj], corpus_key)

['state_of_the_union.txt']

## Vectara RAG (检索增强生成)

现在，我们创建一个 `VectaraQueryConfig` 对象来控制检索和摘要选项：
* 我们启用摘要功能，指定希望 LLM 选择匹配度最高的 7 个块，并用英语进行回复。

使用此配置，让我们创建一个 LangChain `Runnable` 对象，该对象封装了完整的 Vectara RAG 管道，使用了 `as_rag` 方法：

In [6]:
generation_config = GenerationConfig(
    max_used_search_results=7,
    response_language="eng",
    generation_preset_name="vectara-summary-ext-24-05-med-omni",
    enable_factual_consistency_score=True,
)
search_config = SearchConfig(
    corpora=[CorpusConfig(corpus_key=corpus_key)],
    limit=25,
    reranker=ChainReranker(
        rerankers=[
            CustomerSpecificReranker(reranker_id="rnk_272725719", limit=100),
            MmrReranker(diversity_bias=0.2, limit=100),
        ]
    ),
)

config = VectaraQueryConfig(
    search=search_config,
    generation=generation_config,
)

query_str = "what did Biden say?"

rag = vectara.as_rag(config)
rag.invoke(query_str)["answer"]

"President Biden discussed several key issues in his recent statements. He emphasized the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities without masks [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also announced measures against Russian oligarchs, including closing American airspace to Russian flights and targeting their assets, as part of efforts to weaken Russia's economy [3], [7]. Additionally, he reaffirmed the need to protect women's rights, particularly the right to choose as affirmed in Roe v. Wade [5]."

我们也可以像这样使用流式接口：

In [7]:
output = {}
curr_key = None
for chunk in rag.stream(query_str):
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key == "answer":
            print(chunk[key], end="", flush=True)
        curr_key = key

President Biden discussed several key issues in his recent statements. He emphasized the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities without masks [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also announced measures against Russia, including preventing its central bank from defending the Ruble and targeting Russian oligarchs' assets, as part of efforts to weaken Russia's economy and military [3]. Additionally, he reaffirmed the commitment to protect women's rights, particularly the right to choose as affirmed in Roe v. Wade [5]. Lastly, he advocated for funding the police with necessary resources and training to ensure community safety [6].

## 幻觉检测与事实一致性评分

Vectara 推出了 [HHEM](https://huggingface.co/vectara/hallucination_evaluation_model) - 一个开源模型，可用于评估 RAG 响应的事实一致性。

作为 Vectara RAG 的一部分，"事实一致性评分"（FCS），它是开源 HHEM 的改进版本，已通过 API 提供。这会自动包含在 RAG 管道的输出中。

In [8]:
resp = rag.invoke(query_str)
print(resp["answer"])
print(f"Vectara FCS = {resp['fcs']}")

President Biden discussed several key topics in his recent statements. He emphasized the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities without masks [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also announced measures against Russian oligarchs, including closing American airspace to Russian flights and targeting their assets, as part of efforts to weaken Russia's economy [3], [7]. Additionally, he reaffirmed the need to protect women's rights, particularly the right to choose as affirmed in Roe v. Wade [5].
Vectara FCS = 0.61621094


## Vectara 作为 Langchain 的检索器

Vectara 组件也可以用作检索器。

在这种情况下，它的行为与其他任何 LangChain 检索器一样。此模式的主要用途是语义搜索，在此情况下我们禁用摘要功能：

In [9]:
config.generation = None
config.search.limit = 5
retriever = vectara.as_retriever(config=config)
retriever.invoke(query_str)

[Document(metadata={'X-TIKA:Parsed-By': 'org.apache.tika.parser.csv.TextAndCSVParser', 'Content-Encoding': 'UTF-8', 'X-TIKA:detectedEncoding': 'UTF-8', 'X-TIKA:encodingDetector': 'UniversalEncodingDetector', 'Content-Type': 'text/plain; charset=UTF-8', 'source': 'text_file', 'framework': 'langchain'}, page_content='The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs. We are joining with our European allies to find and seize your yachts your luxury apartments your private jets. We are coming for your ill-begotten gains. And tonight I am announcing that we will join our allies in closing off American air space to all Russian flights – further isolating Russia – and adding an additional squeeze –on their economy. The Ruble has lost 30% of its value.'),
 Document(metadata={'X-TIKA:Parsed-By': 'org.apache.tika.parser.csv.TextAndCSVParser', 'Content-Encoding': 'UTF-8', 'X-TIKA:detectedEncoding': 'UTF-8', 'X-TIKA:encodingDetector': '

为了保持向后兼容，您也可以通过检索器启用摘要功能，在这种情况下，摘要将作为附加的 Document 对象添加：

In [10]:
config.generation = GenerationConfig()
config.search.limit = 10
retriever = vectara.as_retriever(config=config)
retriever.invoke(query_str)

[Document(metadata={'X-TIKA:Parsed-By': 'org.apache.tika.parser.csv.TextAndCSVParser', 'Content-Encoding': 'UTF-8', 'X-TIKA:detectedEncoding': 'UTF-8', 'X-TIKA:encodingDetector': 'UniversalEncodingDetector', 'Content-Type': 'text/plain; charset=UTF-8', 'source': 'text_file', 'framework': 'langchain'}, page_content='We won’t be able to compete for the jobs of the 21st Century if we don’t fix that. That’s why it was so important to pass the Bipartisan Infrastructure Law—the most sweeping investment to rebuild America in history. This was a bipartisan effort, and I want to thank the members of both parties who worked to make it happen. We’re done talking about infrastructure weeks. We’re going to have an infrastructure decade.'),
 Document(metadata={'X-TIKA:Parsed-By': 'org.apache.tika.parser.csv.TextAndCSVParser', 'Content-Encoding': 'UTF-8', 'X-TIKA:detectedEncoding': 'UTF-8', 'X-TIKA:encodingDetector': 'UniversalEncodingDetector', 'Content-Type': 'text/plain; charset=UTF-8', 'source': 

## 使用 Vectara 进行高级 LangChain 查询预处理

Vectara 的“RAG 即服务”在创建问答或聊天机器人链方面做了大量繁重的工作。与 LangChain 的集成提供了使用其他功能（如查询预处理）的选项，例如 `SelfQueryRetriever` 或 `MultiQueryRetriever`。让我们来看一个使用 [MultiQueryRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) 的示例。

由于 MQR 使用 LLM，因此我们需要对其进行设置——在这里我们选择 `ChatOpenAI`：

In [14]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
mqr = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)


def get_summary(documents):
    return documents[-1].page_content


(mqr | get_summary).invoke(query_str)

'The remarks made by Biden include his emphasis on the importance of the Bipartisan Infrastructure Law, which he describes as the most significant investment to rebuild America in history. He highlights the bipartisan effort involved in passing this law and expresses gratitude to members of both parties for their collaboration. Biden also mentions the transition from "infrastructure weeks" to an "infrastructure decade" [1]. Additionally, he shares a personal story about his father having to leave their home in Scranton, Pennsylvania, to find work, which influenced his decision to fight for the American Rescue Plan to help those in need [2].'